In [ ]:
# VERSIÓN AVANZADA: COCO Layout Annotations con LayoutParser
import re
import fitz  # PyMuPDF
import numpy as np
from PIL import Image, ImageDraw
import cv2
import os
from dataclasses import dataclass
from typing import List
import layoutparser as lp

# ===== CONFIGURACIÓN PRINCIPAL =====
# Modelo de layout pre-entrenado (COCO Layout)
MODEL_CONFIG = {
    'model_name': 'lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config',
    'model_path': 'lp://PubLayNet/faster_rcnn_R_50_FPN_3x',
    'label_map': {0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"},
    'confidence_threshold': 0.5
}

# Configuración de procesamiento
RENDER_DPI = 200
DEBUG_OUTPUT_DIR = "../data/debug"

# Recorte de márgenes (en pixels)
CROP_TOP = 150
CROP_BOTTOM = 140
CROP_LEFT = 50
CROP_RIGHT = 50

# Crear directorio
os.makedirs(DEBUG_OUTPUT_DIR, exist_ok=True)

@dataclass
class Parrafo:
    texto: str
    bbox: tuple
    page_num: int = 1
    block_type: str = "Text"  # Text, Title, List, Table, Figure
    confidence: float = 0.0

print("🚀 CONFIGURACIÓN COCO LAYOUT:")
print(f"  📊 Modelo: {MODEL_CONFIG['model_name']}")
print(f"  🎯 Confianza mínima: {MODEL_CONFIG['confidence_threshold']}")
print(f"  📁 Debug: {DEBUG_OUTPUT_DIR}")

# Intentar cargar modelo
try:
    print("⏳ Cargando modelo de layout COCO...")
    model = lp.Detectron2LayoutModel(
        MODEL_CONFIG['model_path'], 
        extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", MODEL_CONFIG['confidence_threshold']],
        label_map=MODEL_CONFIG['label_map']
    )
    USE_COCO_MODEL = True
    print("✅ Modelo COCO cargado exitosamente")
except Exception as e:
    print(f"⚠️ No se pudo cargar modelo COCO: {e}")
    print("🔄 Usando detector simple como fallback")
    USE_COCO_MODEL = False


In [28]:
# CONFIGURACIÓN Y IMPORTACIONES
import re
import fitz  # PyMuPDF
import numpy as np
from PIL import Image, ImageDraw
import cv2
import os
from dataclasses import dataclass
from typing import List

# ===== CONFIGURACIÓN PRINCIPAL - AJUSTAR AQUÍ =====
# Parámetros de detección de bloques
MIN_BLOCK_WIDTH = 15     # Ancho mínimo en pixels
MIN_BLOCK_HEIGHT = 15    # Alto mínimo en pixels  
THRESHOLD_VALUE = 200    # Umbral de binarización (0-255)

# Recorte de márgenes (en pixels)
CROP_TOP = 150
CROP_BOTTOM = 140
CROP_LEFT = 50
CROP_RIGHT = 50

# Configuración general
RENDER_DPI = 200
DEBUG_OUTPUT_DIR = "../data/debug"

# Crear directorio y estructura de datos
os.makedirs(DEBUG_OUTPUT_DIR, exist_ok=True)

@dataclass
class Parrafo:
    texto: str
    bbox: tuple
    page_num: int = 1

print(f"✓ Configuración: Bloques min {MIN_BLOCK_WIDTH}x{MIN_BLOCK_HEIGHT}, threshold {THRESHOLD_VALUE}")

✓ Configuración: Bloques min 15x15, threshold 200


In [30]:
# DETECTOR DE LAYOUT CON DEBUG VISUAL
class VisualLayoutDetector:
    def detect_and_visualize(self, image_array, save_debug=True):
        """Detecta bloques de texto y genera visualizaciones de debug"""
        
        # Convertir a escala de grises
        if len(image_array.shape) == 3:
            gray = cv2.cvtColor(image_array, cv2.COLOR_RGB2GRAY)
        else:
            gray = image_array
            
        h, w = gray.shape
        print(f"  📐 Procesando imagen: {w}x{h} pixels")
        
        # PASO 1: Guardar escala de grises
        if save_debug:
            gray_path = os.path.join(DEBUG_OUTPUT_DIR, "step_1_grayscale.png")
            cv2.imwrite(gray_path, gray)
            print(f"  💾 Paso 1: {gray_path}")
        
        # PASO 2: Aplicar umbralización
        _, thresh = cv2.threshold(gray, THRESHOLD_VALUE, 255, cv2.THRESH_BINARY_INV)
        
        if save_debug:
            thresh_path = os.path.join(DEBUG_OUTPUT_DIR, "step_2_threshold.png")
            cv2.imwrite(thresh_path, thresh)
            print(f"  💾 Paso 2: {thresh_path} (threshold={THRESHOLD_VALUE})")
        
        # PASO 3: Encontrar contornos
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        print(f"  🔍 Contornos encontrados: {len(contours)}")
        
        # Analizar contornos
        all_blocks = []
        valid_blocks = []
        
        for contour in contours:
            x, y, w_cont, h_cont = cv2.boundingRect(contour)
            
            block = {
                'bbox': (x, y, x + w_cont, y + h_cont),
                'width': w_cont,
                'height': h_cont,
                'area': w_cont * h_cont,
                'valid': w_cont >= MIN_BLOCK_WIDTH and h_cont >= MIN_BLOCK_HEIGHT
            }
            
            all_blocks.append(block)
            if block['valid']:
                valid_blocks.append(block)
        
        # Estadísticas
        if all_blocks:
            widths = [b['width'] for b in all_blocks]
            heights = [b['height'] for b in all_blocks]
            print(f"  📊 Tamaños - Ancho: {min(widths)}-{max(widths)} | Alto: {min(heights)}-{max(heights)}")
        
        print(f"  ✅ Bloques válidos: {len(valid_blocks)}/{len(all_blocks)}")
        
        # PASO 3: Crear visualización de debug
        if save_debug:
            self._create_debug_visualization(image_array, all_blocks, valid_blocks)
        
        # Ordenar por posición de lectura
        valid_blocks.sort(key=lambda b: (b['bbox'][1], b['bbox'][0]))
        
        return valid_blocks
    
    def _create_debug_visualization(self, original_image, all_blocks, valid_blocks):
        """Crea imagen de debug con contornos marcados"""
        try:
            # Convertir a RGB para dibujar
            if len(original_image.shape) == 3:
                debug_img = Image.fromarray(original_image.astype('uint8'))
            else:
                rgb_array = cv2.cvtColor(original_image.astype('uint8'), cv2.COLOR_GRAY2RGB)
                debug_img = Image.fromarray(rgb_array)
            
            draw = ImageDraw.Draw(debug_img)
            
            # Dibujar contornos rechazados en rojo (muestra limitada)
            rejected = [b for b in all_blocks if not b['valid']]
            for block in rejected[:200]:  # Limitar para no saturar
                draw.rectangle(block['bbox'], outline='red', width=1)
            
            # Dibujar contornos válidos en verde brillante
            for i, block in enumerate(valid_blocks):
                bbox = block['bbox']
                draw.rectangle(bbox, outline='lime', width=3)
                # Número del bloque
                draw.text((bbox[0]+2, bbox[1]+2), str(i+1), fill='lime')
            
            # Información en la esquina
            info = f"Válidos: {len(valid_blocks)} | Rechazados: {len(rejected)} | Config: {MIN_BLOCK_WIDTH}x{MIN_BLOCK_HEIGHT}"
            draw.rectangle((5, 5, 600, 35), fill='black', outline='white')
            draw.text((10, 10), info, fill='white')
            
            # Guardar imagen principal
            debug_path = os.path.join(DEBUG_OUTPUT_DIR, "step_3_contours_debug.png")
            debug_img.save(debug_path)
            print(f"  💾 Paso 3: {debug_path}")
            print(f"       🔴 Rojo: {min(len(rejected), 200)} contornos rechazados")
            print(f"       🟢 Verde: {len(valid_blocks)} contornos válidos")
            
        except Exception as e:
            print(f"  ❌ Error en visualización: {e}")

detector = VisualLayoutDetector()
print("✓ Detector creado")

✓ Detector creado


In [31]:
# FUNCIONES DE PROCESAMIENTO
def page_to_image(page, dpi=RENDER_DPI):
    """Convierte página PDF a imagen PIL"""
    scale = dpi / 72.0
    mat = fitz.Matrix(scale, scale)
    pix = page.get_pixmap(matrix=mat, alpha=False)
    img = Image.frombytes("RGB", (pix.width, pix.height), pix.samples)
    return img, scale

def crop_margins(image):
    """Recorta márgenes de la imagen"""
    width, height = image.size
    left = max(0, CROP_LEFT)
    top = max(0, CROP_TOP)
    right = min(width, width - CROP_RIGHT)
    bottom = min(height, height - CROP_BOTTOM)
    
    if left >= right or top >= bottom:
        print(f"⚠️ Recorte inválido, usando imagen completa")
        return image, 0, 0
    
    cropped = image.crop((left, top, right, bottom))
    print(f"📐 Recorte: {width}x{height} → {cropped.size[0]}x{cropped.size[1]}")
    return cropped, left, top

def extract_text_from_block(pdf_page, bbox, scale, offset_x=0, offset_y=0):
    """Extrae texto de un bloque específico"""
    x1, y1, x2, y2 = bbox
    # Ajustar coordenadas por recorte y escala
    adj_x1 = (x1 + offset_x) / scale
    adj_y1 = (y1 + offset_y) / scale
    adj_x2 = (x2 + offset_x) / scale
    adj_y2 = (y2 + offset_y) / scale
    
    rect = fitz.Rect(adj_x1, adj_y1, adj_x2, adj_y2)
    text = pdf_page.get_textbox(rect).strip()
    
    # Limpiar texto
    text = re.sub(r'\s+', ' ', text)  # Normalizar espacios
    return text

print("✓ Funciones de procesamiento listas")

✓ Funciones de procesamiento listas


In [32]:
# FUNCIÓN PRINCIPAL DE DEBUG
def debug_pdf_paragraphs(pdf_path):
    """Analiza la primera página de un PDF y genera debug visual"""
    print(f"🔍 ANÁLISIS DE PÁRRAFOS: {os.path.basename(pdf_path)}")
    print("=" * 70)
    
    try:
        # Abrir PDF
        doc = fitz.open(pdf_path)
        page = doc[0]
        print(f"✓ PDF cargado: {len(doc)} páginas")
        
        # Convertir a imagen
        pil_img, scale = page_to_image(page)
        print(f"✓ Imagen generada: {pil_img.size} (escala: {scale:.2f})")
        
        # Guardar original
        orig_path = os.path.join(DEBUG_OUTPUT_DIR, "page_1_original.png")
        pil_img.save(orig_path)
        print(f"✓ Original guardada: {orig_path}")
        
        # Recortar márgenes
        cropped_img, offset_x, offset_y = crop_margins(pil_img)
        crop_path = os.path.join(DEBUG_OUTPUT_DIR, "page_1_cropped.png")
        cropped_img.save(crop_path)
        print(f"✓ Recortada guardada: {crop_path}")
        
        # Detectar bloques con visualización
        print(f"\n🔍 DETECCIÓN DE BLOQUES:")
        image_array = np.array(cropped_img)
        blocks = detector.detect_and_visualize(image_array, save_debug=True)
        
        # Extraer texto de los primeros bloques válidos
        print(f"\n📝 EXTRACCIÓN DE TEXTO (primeros 5 bloques):")
        paragraphs = []
        
        for i, block in enumerate(blocks[:5], 1):
            bbox = block['bbox']
            text = extract_text_from_block(page, bbox, scale, offset_x, offset_y)
            
            print(f"\n  BLOQUE {i}: {block['width']}x{block['height']}px")
            if text and len(text.strip()) > 3:
                preview = text[:120] + '...' if len(text) > 120 else text
                print(f"    ✅ \"{preview}\"")
                paragraphs.append(Parrafo(texto=text, bbox=bbox, page_num=1))
            else:
                print(f"    ❌ Sin texto válido")
        
        doc.close()
        
        # Resumen final
        print(f"\n📊 RESUMEN FINAL:")
        print(f"  • Bloques detectados: {len(blocks)}")
        print(f"  • Párrafos con texto: {len(paragraphs)}")
        print(f"  • Configuración: min {MIN_BLOCK_WIDTH}x{MIN_BLOCK_HEIGHT}px, threshold {THRESHOLD_VALUE}")
        print(f"\n📁 Revisar archivos en: {DEBUG_OUTPUT_DIR}/")
        print(f"  → step_3_contours_debug.png (¡IMAGEN PRINCIPAL!)")
        
        return paragraphs, blocks
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return [], []

print("✓ Función principal lista")

✓ Función principal lista


In [33]:
# EJECUCIÓN AUTOMÁTICA
print("🚀 INICIANDO ANÁLISIS AUTOMÁTICO")
print("=" * 50)

# Buscar PDFs en el proyecto
project_root = "/Users/alexa/Projects/cdmx_kg"
search_paths = [
    os.path.join(project_root, "Mexico_City", "laws"),
    os.path.join(project_root, "Mexico_City", "laws_1"),
    project_root
]

pdf_files = []
for search_path in search_paths:
    if os.path.exists(search_path):
        for root, dirs, files in os.walk(search_path):
            for file in files:
                if file.endswith('.pdf'):
                    pdf_files.append(os.path.join(root, file))

if pdf_files:
    pdf_path = pdf_files[0]
    print(f"📄 Analizando: {os.path.basename(pdf_path)}")
    print(f"📍 Ruta: {pdf_path}")
    print()
    
    # Ejecutar análisis
    paragraphs, blocks = debug_pdf_paragraphs(pdf_path)
    
    # Instrucciones finales
    print(f"\n" + "="*70)
    print(f"🎯 SIGUIENTE PASO:")
    if len(blocks) == 0:
        print(f"   ❌ No se detectaron bloques válidos")
        print(f"   💡 Ajustar parámetros más permisivos:")
        print(f"      MIN_BLOCK_WIDTH = 20")
        print(f"      MIN_BLOCK_HEIGHT = 10") 
        print(f"      THRESHOLD_VALUE = 150")
    else:
        print(f"   ✅ Abrir: {DEBUG_OUTPUT_DIR}/step_3_contours_debug.png")
        print(f"   🔍 Verificar que las cajas VERDES cubren el texto")
        print(f"   ⚙️ Si no, ajustar parámetros en la celda 1 y re-ejecutar")
    
else:
    print("❌ No se encontraron archivos PDF en el proyecto")
    print("💡 Verificar que hay PDFs en:")
    for path in search_paths:
        print(f"   - {path}")

🚀 INICIANDO ANÁLISIS AUTOMÁTICO
📄 Analizando: LEY_DE_EDUCACION_DE_LA_CDMX_3.4.pdf
📍 Ruta: /Users/alexa/Projects/cdmx_kg/pdfs/LEY_DE_EDUCACION_DE_LA_CDMX_3.4.pdf

🔍 ANÁLISIS DE PÁRRAFOS: LEY_DE_EDUCACION_DE_LA_CDMX_3.4.pdf
✓ PDF cargado: 25 páginas
✓ Imagen generada: (1700, 2200) (escala: 2.78)
✓ Original guardada: ../data/debug/page_1_original.png
📐 Recorte: 1700x2200 → 1600x1910
✓ Recortada guardada: ../data/debug/page_1_cropped.png

🔍 DETECCIÓN DE BLOQUES:
  📐 Procesando imagen: 1600x1910 pixels
  💾 Paso 1: ../data/debug/step_1_grayscale.png
  💾 Paso 2: ../data/debug/step_2_threshold.png (threshold=200)
  🔍 Contornos encontrados: 1778
  📊 Tamaños - Ancho: 3-786 | Alto: 1-26
  ✅ Bloques válidos: 257/1778
  💾 Paso 3: ../data/debug/step_3_contours_debug.png
       🔴 Rojo: 200 contornos rechazados
       🟢 Verde: 257 contornos válidos

📝 EXTRACCIÓN DE TEXTO (primeros 5 bloques):

  BLOQUE 1: 16x22px
    ❌ Sin texto válido

  BLOQUE 2: 16x22px
    ❌ Sin texto válido

  BLOQUE 3: 16x22px
 